In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!unzip gdrive/My\ Drive/ComputerVision/train.zip


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf 
from PIL import Image
import os 
from skimage.filters import threshold_otsu
from sklearn.model_selection import train_test_split 
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout 
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical

In [ ]:
data = []
labels = []
classes = 43 
cur_path = os.getcwd()
cur_path

'/content'

In [ ]:
for i in range(classes): 
  path = os. path.join(cur_path,'train', str(i)) 
  images = os.listdir(path) 
  for a in images: 
     try:
      image = cv2.imread(path + '/'+ a) 
      image = cv2.resize(image,(30,30))
      #image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
      #image=cv2.equalizeHist(image)
      image = np.array(image)
      data.append(image)
      labels.append(i)
     except: 
        print("Error loading image") 
data = np.array(data)
labels = np.array(labels)


In [ ]:
X_t1, X_t2, y_t1, y_t2 = train_test_split(data, labels, test_size=0.2, random_state=42)
print(X_t1.shape, X_t2.shape, y_t1.shape, y_t2.shape)
#Converting the labels into one hot encoding
y_t1 = to_categorical(y_t1, 43)
y_t2 = to_categorical(y_t2, 43)

(31367, 30, 30, 3) (7842, 30, 30, 3) (31367,) (7842,)


In [ ]:
"""
def filter_image(image, mask):
    r = image[:,:,0] * mask
    g = image[:,:,1] * mask
    b = image[:,:,2] * mask
    return np.dstack([r,g,b])
"""

In [ ]:
#a function that applies thresholding to the images, but it turned out not to be beneficial to the end result
"""
def thresholding(images):
    final_images=[]
    for i in range(0,len(images)):
        img=images[i]
        img_rgb=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img_gray=cv2.cvtColor(img_rgb,cv2.COLOR_RGB2GRAY)
        thresh = threshold_otsu(img_gray)
        img_otsu  = img_gray < thresh
        filtered = filter_image(img, img_otsu)
        final_images.append(filtered)
    return final_images
    """

In [ ]:
"""
final_images=thresholding(data)
final_images = np.array(final_images)
X_t1, X_t2, y_t1, y_t2 = train_test_split(final_images, labels, test_size=0.2, random_state=42)
y_t1 = to_categorical(y_t1, 43)
y_t2 = to_categorical(y_t2, 43)
"""

In [ ]:
#Building the model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=X_t1.shape[1:]))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
#model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))
#Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
eps = 20
anc = model.fit(X_t1, y_t1, batch_size=32, epochs=eps, validation_data=(X_t2, y_t2))

Epoch 1/20
981/981 [==============================] - 33s 33ms/step - loss: 2.5949 - accuracy: 0.3891 - val_loss: 0.9048 - val_accuracy: 0.7735
Epoch 2/20
981/981 [==============================] - 32s 33ms/step - loss: 0.6087 - accuracy: 0.8254 - val_loss: 0.2833 - val_accuracy: 0.9260
Epoch 3/20
981/981 [==============================] - 33s 34ms/step - loss: 0.3474 - accuracy: 0.9015 - val_loss: 0.1448 - val_accuracy: 0.9577
Epoch 4/20
981/981 [==============================] - 32s 33ms/step - loss: 0.2967 - accuracy: 0.9155 - val_loss: 0.1289 - val_accuracy: 0.9649
Epoch 5/20
981/981 [==============================] - 33s 34ms/step - loss: 0.2524 - accuracy: 0.9319 - val_loss: 0.1299 - val_accuracy: 0.9670
Epoch 6/20
981/981 [==============================] - 32s 33ms/step - loss: 0.2290 - accuracy: 0.9352 - val_loss: 0.1494 - val_accuracy: 0.9690
Epoch 7/20
981/981 [==============================] - 32s 32ms/step - loss: 0.2307 - accuracy: 0.9383 - val_loss: 0.1250 - val_accuracy:

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
rounded_labels=np.argmax(y_t2, axis=1)

In [ ]:
pred = np.argmax(model.predict(X_t2), axis=-1)
print(accuracy_score(rounded_labels, pred))

0.9813823004335629
